In [ ]:
from scipy.signal import butter, lfilter, fftconvolve

def create_system_response(excitation, fs, noise_std=0.12):
    # Design a Butterworth filter
    B, A = butter(10, [0.1, 0.7], fs=fs)
    
    # Pass the excitation signal through the filter
    system_response = lfilter(B, A, excitation)
    
    # Add noise to the system response
    noisy_response = system_response + noise_std * np.random.randn(*system_response.shape)
    
    return noisy_response, B, A

# Generate the system response
system_response, B, A = create_system_response(sweep, fs)


In [ ]:
plt.figure()
plt.plot(system_response)
plt.title('System Response')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
plt.grid(True)
plt.show()

def estimate_impulse_response(excitation, response, fs):
    # Estimate the impulse response using the deconvolution
    impulse_response, remainder = fftconvolve(response, excitation[::-1], mode='full')[:len(excitation)]
    
    # Normalize the impulse response
    impulse_response /= np.max(np.abs(impulse_response))
    
    return impulse_response

# Estimate the impulse response
estimated_ir = estimate_impulse_response(sweep, system_response, fs)

# Get the true impulse response of the system
true_ir = lfilter([1], B, A, zi=np.zeros(A.shape[0]-1))[0]

# Plot the true and estimated impulse responses
plt.figure()
plt.plot(estimated_ir, label='Estimated impulse response')
plt.plot(true_ir, 'ro', label='True impulse response')
plt.legend()
plt.title('True and Estimated Impulse Responses')
plt.xlabel('Sample')
plt.ylabel('Amplitude')
plt.grid(True)
plt.show()